## Demo: ParallelActivity
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

This notebook provides an example of a parallel activity. The parallel activity initiate a set of sub processes simultaneously.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes
In this example we won't use object classes

#### 3. Create objects
##### 3.1. Create site object(s)
No site objects are created

##### 3.2. Create vessel object(s)
No vessel objects are created

##### 3.3 Create activity/activities

In [3]:
# initialise registry
registry = {}

In [4]:
# create a reporting activity
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    registry=registry,
    duration=0,
)

# create a list of the sub processes
sub_processes = [
    model.BasicActivity(
        env=my_env,
        name="Basic activity 1",
        registry=registry,
        duration=14,
        additional_logs=[reporting_activity],
    ),
    model.BasicActivity(
        env=my_env,
        name="Basic activity 2",
        registry=registry,
        duration=5,
        additional_logs=[reporting_activity],
    ),
    model.BasicActivity(
        env=my_env,
        name="Basic activity 3",
        registry=registry,
        duration=220,
        additional_logs=[reporting_activity],
    ),
]

# create a parallel activity
parallel_activity = model.ParallelActivity(
    env= my_env,
    name= "Parallel process",
    registry= registry,
    sub_processes= sub_processes,
)

#### 4. Register processes and run simpy

In [5]:
# initate the simpy processes defined in the 'parallel activity' and run simpy
model.register_processes([parallel_activity, reporting_activity])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs

In [6]:
display(plot.get_log_dataframe(reporting_activity, [*sub_processes, parallel_activity, reporting_activity]))

,Activity,Timestamp,ActivityState,type,ref
0,Reporting activity,1970-01-01 00:00:00,START,NaN,NaN
1,Reporting activity,1970-01-01 00:00:00,STOP,NaN,NaN
2,Basic activity 1,1970-01-01 00:00:00,START,additional log,457d9e4c-6085-4c6a-8dc6-b752953da668
3,Basic activity 2,1970-01-01 00:00:00,START,additional log,45f39d79-f6e1-4c3b-b5c5-1429811f5cb5
4,Basic activity 3,1970-01-01 00:00:00,START,additional log,f55aca52-a852-4943-b52a-b163b987ff6d
5,Basic activity 2,1970-01-01 00:00:05,STOP,additional log,45f39d79-f6e1-4c3b-b5c5-1429811f5cb5
6,Basic activity 1,1970-01-01 00:00:14,STOP,additional log,457d9e4c-6085-4c6a-8dc6-b752953da668
7,Basic activity 3,1970-01-01 00:03:40,STOP,additional log,f55aca52-a852-4943-b52a-b163b987ff6d


In [7]:
for act in [*sub_processes, parallel_activity]:
    display(plot.get_log_dataframe(act, [*sub_processes, parallel_activity, reporting_activity]))

,Activity,Timestamp,ActivityState
0,Basic activity 1,1970-01-01 00:00:00,START
1,Basic activity 1,1970-01-01 00:00:14,STOP


,Activity,Timestamp,ActivityState
0,Basic activity 2,1970-01-01 00:00:00,START
1,Basic activity 2,1970-01-01 00:00:05,STOP


,Activity,Timestamp,ActivityState
0,Basic activity 3,1970-01-01 00:00:00,START
1,Basic activity 3,1970-01-01 00:03:40,STOP


,Activity,Timestamp,ActivityState,type,ref
0,Parallel process,1970-01-01 00:00:00,START,NaN,NaN
1,Parallel process,1970-01-01 00:00:00,START,subprocess,457d9e4c-6085-4c6a-8dc6-b752953da668
2,Parallel process,1970-01-01 00:00:00,START,subprocess,45f39d79-f6e1-4c3b-b5c5-1429811f5cb5
3,Parallel process,1970-01-01 00:00:00,START,subprocess,f55aca52-a852-4943-b52a-b163b987ff6d
4,Parallel process,1970-01-01 00:00:05,STOP,subprocess,45f39d79-f6e1-4c3b-b5c5-1429811f5cb5
5,Parallel process,1970-01-01 00:00:14,STOP,subprocess,457d9e4c-6085-4c6a-8dc6-b752953da668
6,Parallel process,1970-01-01 00:03:40,STOP,subprocess,f55aca52-a852-4943-b52a-b163b987ff6d
7,Parallel process,1970-01-01 00:03:40,STOP,NaN,NaN


##### 5.2 Visualise gantt charts

In [8]:
plot.get_gantt_chart([parallel_activity, *sub_processes])